En este proceso vamos a Tomar los Datasets limpios que generamos en la primer parte (1-ETL-EDA.ipynb) y vamos a generar un nuevo dataset, pero solamente con los datos necesarios para responder a las consultas que se nos pidio en las funciones.

Estos datasets nuevos que generamos para cada consulta, van a ser mas livianos ya que van a estar agrupados, para contestar lo que la funcion nos pide y poder subirlos sin problemas a github y utilizarlos para la API

Voy a preparar un .csv para cada consulta que tenemos que hacer

Primer Funcion de consulta

1.  def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.

Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

Como vemos por la funcion que tenemos, se nos pide los datos de las columnas:

Releasedate (se encuentra en output_games)

Genres (se encuentra en output_games) 

PlaytimeForever (se encuentra en user_items)

Vamos a preparar el primer CSV para la funcion PlayTimeGenre

In [2]:
# Cargamos las librerias que vamos a utilizar
import pandas as pd
import os
import ast
from ast import literal_eval
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

In [3]:
# creamos las variables para las Rutas relativas a los archivos csv
ruta_AUS_User_Items = os.path.join('Datasets-Limpios', 'user_items.parquet')
ruta_Output_Steam_Games = os.path.join('Datasets-Limpios', 'output_games.parquet')

In [4]:
# Leemos el csv y creamos un nuevo dataframe
output_games = pd.read_parquet(ruta_Output_Steam_Games)

In [5]:
# vemos output_games
output_games.head(1)

,Genres,ItemName,ReleaseDate,Price,ItemId,Developer,ReleaseYear
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018.0


In [6]:
# Vemos la cantidad de registros que tiene output_games
len(output_games)

31994

In [7]:
# Leemos el csv de user_items.csv
user_items = pd.read_parquet(ruta_AUS_User_Items)

In [8]:
# lo vemos
user_items.head(1)

,ItemId,ItemName,PlaytimeForever,UserId,ItemsCount
0,10,Counter-Strike,6,76561197970982479,277


In [9]:
# vemos la cantidad que tenemos en user_items.csv
len(user_items)

5094092

Ahora vamos a crear el nuevo dataset con las columnas que precisamos para la consulta

In [10]:
# Seleccionamos las columnas de cada DataFrame
columnas_file1 = user_items[['PlaytimeForever','ItemId']]
columnas_file2 = output_games[['Genres', 'ReleaseDate','ItemId']]
# Traemos la columna ItemId para hacer la referencia entre las tablas

Hacemos un MERGE de las Tablas user_items y output_games

In [11]:
# Merge basado en el campo ItemId
merged_data = pd.merge(columnas_file1, columnas_file2, left_on='ItemId', right_on='ItemId', how='inner')

Ahora tenemos un nuevo Dataframe con las columnas que seleccionamos solamente

In [12]:
# Chequeamos el nuevo Dataset
merged_data.head(1)

,PlaytimeForever,ItemId,Genres,ReleaseDate
0,6,10,[Action],2000-11-01


In [13]:
# Cantidad de registros en el nuevo Dataframe
len(merged_data)

4244767

In [14]:
# Vemos los nulos que quedaron en el nuevo dataframe
merged_data.isnull().sum()

PlaytimeForever         0
ItemId                  0
Genres                  0
ReleaseDate        101415
dtype: int64

Tratamos los nulos que nos dejo hacer el merge de las tablas, quedando con 101415 nulos, que se pueden tratar borrandolos ya que no representan un valor alto respecto a 4143352 de datos que se tienen.

Tratamos los nulos

In [15]:
# Borraros los nulos que tenemos
merged_data = merged_data.dropna(subset=['ReleaseDate'])

In [16]:
# Chequeamos los nulos nuevamente
merged_data.isnull().sum()

PlaytimeForever    0
ItemId             0
Genres             0
ReleaseDate        0
dtype: int64

In [17]:
# Chequeamos con cuanta cantidad de datos nos quedamos despues del dropna
len(merged_data)

4143352

Antes de empezar a agrupar chequeamos como quedaron los tipos de datos en el nuevo Dataframe

In [18]:
# Vemos el tipo de dato para saber que tipo de dato tiene la columna ReleaseDate
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4143352 entries, 0 to 4244765
Data columns (total 4 columns):
 #   Column           Dtype         
---  ------           -----         
 0   PlaytimeForever  int32         
 1   ItemId           int32         
 2   Genres           object        
 3   ReleaseDate      datetime64[ns]
dtypes: datetime64[ns](1), int32(2), object(1)
memory usage: 126.4+ MB


In [19]:
# Cambiamos el tipo de dato del campo ReleaseDate a formato Datetime de pandas, solamente con el año que es lo que
# necesitamos para la consulta
merged_data['ReleaseDate'] = pd.to_datetime(merged_data['ReleaseDate'], errors='coerce', format='%Y', exact=False)

In [20]:
# Vemos el tipo de dato para saber que tipo de dato tiene la columna ReleaseDate
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4143352 entries, 0 to 4244765
Data columns (total 4 columns):
 #   Column           Dtype         
---  ------           -----         
 0   PlaytimeForever  int32         
 1   ItemId           int32         
 2   Genres           object        
 3   ReleaseDate      datetime64[ns]
dtypes: datetime64[ns](1), int32(2), object(1)
memory usage: 126.4+ MB


In [21]:
# Aplicamos explode a la columna de géneros para separar en filas la lista dentro del campo Genres
merged_data = merged_data.explode('Genres')

In [22]:
merged_data['Genres'].unique()

array(['Action', 'Strategy', 'RPG', 'Indie', 'Casual', 'Simulation',
       'Adventure', 'Racing', 'Sports', 'Early Access', 'Free to Play',
       'Massively Multiplayer', nan, 'Utilities',
       'Animation &amp; Modeling', 'Design &amp; Illustration',
       'Education', 'Web Publishing', 'Video Production',
       'Audio Production', 'Software Training', 'Photo Editing'],
      dtype=object)

In [23]:
merged_data.head(1)

,PlaytimeForever,ItemId,Genres,ReleaseDate
0,6,10,Action,2000-11-01


Ahora vamos a agrupar por Genres y ReleaseDate, sumando la cantidad de horas jugadas.

In [24]:
# Agrupamos por Género, Fecha de Lanzamiento y sumar las horas jugadas
grouped_data = merged_data.groupby(['Genres', merged_data['ReleaseDate'].dt.year])['PlaytimeForever'].sum().reset_index()

In [25]:
# Chequeamos como nos quedo la agrupacion que hicimos
grouped_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Genres           343 non-null    object
 1   ReleaseDate      343 non-null    int64 
 2   PlaytimeForever  343 non-null    int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 6.8+ KB


In [26]:
# Chequeamos como nos quedo con el agrupado
grouped_data.tail()

,Genres,ReleaseDate,PlaytimeForever
338,Web Publishing,2013,333678
339,Web Publishing,2014,33641
340,Web Publishing,2015,348673
341,Web Publishing,2016,136
342,Web Publishing,2017,9382


In [27]:
# Chequeamos nulos despues de desanidar las listas de la columna Genres
grouped_data.Genres.isnull().sum()

0

In [28]:
# Chequeamos los vacios que vimos anteriormente para reemplazarlos por nan nulos para borrarlos
grouped_data[grouped_data.Genres == ''].count()

Genres             0
ReleaseDate        0
PlaytimeForever    0
dtype: int64

In [29]:
# Mostramos como quedo nuestro Dataframe
grouped_data.head()

,Genres,ReleaseDate,PlaytimeForever
0,Action,1983,3473
1,Action,1984,384
2,Action,1988,16001
3,Action,1989,607
4,Action,1990,18335


In [30]:
grouped_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Genres           343 non-null    object
 1   ReleaseDate      343 non-null    int64 
 2   PlaytimeForever  343 non-null    int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 6.8+ KB


In [31]:
# Chequeamos la cantidad de datos que nos quedaron
len(grouped_data)

343

In [32]:
# Vemos como quedaron los generos unicos
grouped_data['Genres'].unique()

array(['Action', 'Adventure', 'Animation &amp; Modeling',
       'Audio Production', 'Casual', 'Design &amp; Illustration',
       'Early Access', 'Education', 'Free to Play', 'Indie',
       'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing',
       'Simulation', 'Software Training', 'Sports', 'Strategy',
       'Utilities', 'Video Production', 'Web Publishing'], dtype=object)

In [33]:
# como vemos hay unos errores de caracteres, que vamos a solucionar de la siguiente manera
grouped_data['Genres'] = grouped_data['Genres'].replace('Design &amp; Illustration', 'Design & Illustration')
grouped_data['Genres'] = grouped_data['Genres'].replace('Animation &amp; Modeling', 'Animation & Modeling')

In [34]:
# Verificamos los valores únicos después del reemplazo
print(grouped_data['Genres'].unique())

['Action' 'Adventure' 'Animation & Modeling' 'Audio Production' 'Casual'
 'Design & Illustration' 'Early Access' 'Education' 'Free to Play' 'Indie'
 'Massively Multiplayer' 'Photo Editing' 'RPG' 'Racing' 'Simulation'
 'Software Training' 'Sports' 'Strategy' 'Utilities' 'Video Production'
 'Web Publishing']


Como Ultimo paso guardamos el dataframe ya tratado para la funcion de consulta

In [35]:
# Guardamos el resultado en un nuevo archivo CSV
grouped_data.to_csv('consulta1.csv', index=False)

---------------------------------------------------------------------------------------------

2.    def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

Para esta funcion vamos a precisar las siguientes columnas:

UserId (se encuentra en user_items)

plyatimeforever (se encuentra en el dataset user_items)

Genre (se encuentra en el dataset output_games)

ReleaseDate (se encuentra en el dataset output_games)

In [36]:
# creamos las variables para las Rutas relativas a los archivos csv
ruta_AUS_User_Items = os.path.join('Datasets-Limpios', 'user_items.parquet')
ruta_Output_Steam_Games = os.path.join('Datasets-Limpios', 'output_games.parquet')

In [37]:
# Creamos los dataframes
output_games = pd.read_parquet(ruta_Output_Steam_Games)
user_items = pd.read_parquet(ruta_AUS_User_Items)

In [38]:
# Chequeamos las columnas y el contenido del dataframe
output_games.head()

,Genres,ItemName,ReleaseDate,Price,ItemId,Developer,ReleaseYear
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018.0
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,0.00,643980,Secret Level SRL,2018.0
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,0.00,670290,Poolians.com,2017.0
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域,2017.0
4,"[Action, Indie, Casual, Sports]",Log Challenge,NaT,2.99,773570,Desconocido,NaN


In [39]:
# Chequeamos las columnas y el contenido del dataframe
user_items.head()

,ItemId,ItemName,PlaytimeForever,UserId,ItemsCount
0,10,Counter-Strike,6,76561197970982479,277
1,20,Team Fortress Classic,0,76561197970982479,277
2,30,Day of Defeat,7,76561197970982479,277
3,40,Deathmatch Classic,0,76561197970982479,277
4,50,Half-Life: Opposing Force,0,76561197970982479,277


In [40]:
# Vamos a Quitar las columnas que no vamos a utilizar

#Para la tabla output_games
primero = output_games.drop(['ItemName', 'Developer', 'Price'],axis=1)
#Para la tabla user_items
segundo = user_items.drop(['ItemName','ItemsCount'],axis=1)

# Hacemos el merge de las tablas
merged_data=pd.merge(segundo,primero,left_on='ItemId', right_on='ItemId', how='right')
merged_data.drop(['ItemId'], axis=1, inplace=True)

In [41]:
# Chequeamos el dataframe creado con el merge
merged_data.head()

,PlaytimeForever,UserId,Genres,ReleaseDate,ReleaseYear
0,NaN,NaN,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04,2018.0
1,NaN,NaN,"[Free to Play, Indie, RPG, Strategy]",2018-01-04,2018.0
2,NaN,NaN,"[Casual, Free to Play, Indie, Simulation, Sports]",2017-07-24,2017.0
3,NaN,NaN,"[Action, Adventure, Casual]",2017-12-07,2017.0
4,NaN,NaN,"[Action, Indie, Casual, Sports]",NaT,NaN


In [42]:
len(merged_data)

4267571

In [43]:
# Chequeamos si hay nulos en el dataframe que creamos
merged_data.isnull().sum()

PlaytimeForever     22804
UserId              22804
Genres                  0
ReleaseDate        103291
ReleaseYear        103291
dtype: int64

Vamos a tratar los Nulos

In [44]:
# Analizamos que el merge quedo con algunas filas que no coinciden con el dataset ya que hay mas informacion en
# uno mas que en el otro
# se van a limpiar los nulos
merged_data.dropna(inplace=True)

In [45]:
# Chequeamos como quedo el dataset con los nulos dropeados
merged_data.isnull().sum()

PlaytimeForever    0
UserId             0
Genres             0
ReleaseDate        0
ReleaseYear        0
dtype: int64

In [46]:
# Cantidad de datos en el dataframe
len(merged_data)

4143352

In [47]:
# Vemos el dataframe
merged_data.head()

,PlaytimeForever,UserId,Genres,ReleaseDate,ReleaseYear
27,5.0,UTNerd24,"[Action, Indie, Racing]",1997-06-30,1997.0
28,0.0,I_DID_911_JUST_SAYING,"[Action, Indie, Racing]",1997-06-30,1997.0
29,0.0,76561197962104795,"[Action, Indie, Racing]",1997-06-30,1997.0
30,0.0,r3ap3r78,"[Action, Indie, Racing]",1997-06-30,1997.0
31,13.0,saint556,"[Action, Indie, Racing]",1997-06-30,1997.0


In [48]:
# Como se ve en el head() de merged_data, se cambio  el tipo de dato de int a float, 
# vamos a modificarlo nuevamente
merged_data['PlaytimeForever'] = merged_data['PlaytimeForever'].astype(int)
# Tratamos la columna de ReleaseDate para que solo traiga el año
merged_data['ReleaseDate'] = pd.to_datetime(merged_data['ReleaseDate'], errors='coerce').dt.year

In [49]:
merged_data.head()

,PlaytimeForever,UserId,Genres,ReleaseDate,ReleaseYear
27,5,UTNerd24,"[Action, Indie, Racing]",1997,1997.0
28,0,I_DID_911_JUST_SAYING,"[Action, Indie, Racing]",1997,1997.0
29,0,76561197962104795,"[Action, Indie, Racing]",1997,1997.0
30,0,r3ap3r78,"[Action, Indie, Racing]",1997,1997.0
31,13,saint556,"[Action, Indie, Racing]",1997,1997.0


In [50]:
# Aplicamos explode a la columna de géneros
merged_data = merged_data.explode('Genres')

In [51]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9840519 entries, 27 to 4267553
Data columns (total 5 columns):
 #   Column           Dtype  
---  ------           -----  
 0   PlaytimeForever  int32  
 1   UserId           object 
 2   Genres           object 
 3   ReleaseDate      int64  
 4   ReleaseYear      float64
dtypes: float64(1), int32(1), int64(1), object(2)
memory usage: 412.9+ MB


In [52]:
merged_data.head()

,PlaytimeForever,UserId,Genres,ReleaseDate,ReleaseYear
27,5,UTNerd24,Action,1997,1997.0
27,5,UTNerd24,Indie,1997,1997.0
27,5,UTNerd24,Racing,1997,1997.0
28,0,I_DID_911_JUST_SAYING,Action,1997,1997.0
28,0,I_DID_911_JUST_SAYING,Indie,1997,1997.0


In [53]:
# Vamos a agrupar por los jugadores con más horas jugadas por género y año
max_hours_df = merged_data.loc[merged_data.groupby(['Genres', 'ReleaseDate'])['PlaytimeForever'].idxmax()]

# Filtramos las columnas necesarias
max_hours_df = max_hours_df[['Genres', 'ReleaseDate', 'UserId', 'PlaytimeForever']]

# Segunda agrupación para obtener las horas jugadas por año para el usuario y la fecha
consulta2 = max_hours_df.groupby(['Genres', 'ReleaseDate', 'UserId'])['PlaytimeForever'].sum().reset_index()

In [54]:
consulta2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Genres           573 non-null    object
 1   ReleaseDate      573 non-null    int64 
 2   UserId           573 non-null    object
 3   PlaytimeForever  573 non-null    int32 
dtypes: int32(1), int64(1), object(2)
memory usage: 15.8+ KB


In [55]:
consulta2.head(20).sort_values('PlaytimeForever',ascending=False)

,Genres,ReleaseDate,UserId,PlaytimeForever
15,Action,1995,Steamified,14670
14,Action,1995,CorporalDucky,10192
17,Action,1996,Steamified,7478
9,Action,1994,Cow666,5539
19,Action,1997,76561197972166311,2632
5,Action,1991,76561198056997935,2283
8,Action,1993,spooke512,1812
4,Action,1990,76561198041356854,1424
10,Action,1994,Prankzter,1423
0,Action,1983,76561197966936422,993


Exportamos la consulta

In [56]:
# Guardamos el resultado en un nuevo archivo CSV
consulta2.to_csv('consulta2.csv', index=False)

--------------------------------------------------------------

3.    def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Como vemos por la funcion que tenemos, se nos pide los datos de las columnas:

YearPosted (user_reviews)

ItemId (user_reviews)

ItemName (output_games)

recommend (user_reviews)

Review (user_reviews)

UserId (user_reviews)

In [57]:
# Cargamos con ruta relativa el csv
ruta_AUS_User_Reviews = os.path.join('Datasets-Limpios', 'user_reviews.parquet')
ruta_Output_Steam_Games = os.path.join('Datasets-Limpios', 'output_games.parquet')

In [58]:
user_reviews = pd.read_parquet(ruta_AUS_User_Reviews)
output_games = pd.read_parquet(ruta_Output_Steam_Games)

In [59]:
user_reviews.columns

Index(['ItemId', 'Recommend', 'Review', 'UserId', 'YearPosted'], dtype='object')

In [60]:
user_reviews.head()

,ItemId,Recommend,Review,UserId,YearPosted
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011
1,22200,True,It's unique and worth a playthrough.,76561197970982479,2011
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2011
3,251610,True,I know what you think when you see this title ...,js41637,2014
4,227300,True,For a simple (it's actually not all that simpl...,js41637,2013


Lo primero que vamos a hacer es analizar la columna review y hacer un analisis de sentimiento

En el dataset user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. Debes crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

In [61]:
# Descarga los recursos necesarios para nltk
nltk.download('vader_lexicon')

# Función para realizar el análisis de sentimientos
def analyze_sentiment(review):
    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(review)['compound']
    
    if sentiment_score >= 0.05:
        return 2  # Positivo
    elif sentiment_score <= -0.05:
        return 0  # Malo
    else:
        return 1  # Neutral

# Aplicamos el análisis de sentimientos y creamos la nueva columna 'sentiment_analysis'
user_reviews['sentiment_analysis'] = user_reviews['Review'].apply(lambda x: analyze_sentiment(x) if pd.notnull(x) else 1)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [62]:
# Vemos el dataset con la nueva columna creada
user_reviews[user_reviews['sentiment_analysis'] == 2]

,ItemId,Recommend,Review,UserId,YearPosted,sentiment_analysis
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011,2
1,22200,True,It's unique and worth a playthrough.,76561197970982479,2011,2
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2011,2
3,251610,True,I know what you think when you see this title ...,js41637,2014,2
4,227300,True,For a simple (it's actually not all that simpl...,js41637,2013,2
...,...,...,...,...,...,...
58426,70,True,a must have classic from steam definitely wort...,76561198312638244,2016,2
58427,362890,True,this game is a perfect remake of the original ...,76561198312638244,2016,2
58428,273110,True,had so much fun plaing this and collecting res...,LydiaMorley,2016,2
58429,730,True,:D,LydiaMorley,2016,2


In [63]:
user_reviews[user_reviews['sentiment_analysis'] == 1]

,ItemId,Recommend,Review,UserId,YearPosted,sentiment_analysis
9,263360,True,"Random drops and random quests, with stat poin...",evcentric,2014,1
18,211420,True,Git gud,maplemage,2014,1
22,249130,True,This game is Marvellous.,Wackky,2014,1
23,207610,True,"It reminds me of that TV Show called ""The Walk...",Wackky,2012,1
27,730,True,ZIKA DO BAILE,76561198079601835,2016,1
...,...,...,...,...,...,...
58407,220090,True,GET THIS GAME AND CHAPTER TWO!!!!!!!!!!!!!!!!!...,MeloncraftLP,2016,1
58408,262850,True,THIS GAME!!!!!!!!!!!!!!!!!! WOOOOOOOOOOOOOOOOO...,MeloncraftLP,2016,1
58411,298110,False,"uplay, everytime",943525,2016,1
58418,427730,True,dont ask,sexyawp,2016,1


In [64]:
user_reviews[user_reviews['sentiment_analysis'] == 0]

,ItemId,Recommend,Review,UserId,YearPosted,sentiment_analysis
16,207610,True,The ending to this game is.... ♥♥♥♥♥♥♥.... Jus...,doctr,2012,0
29,72850,True,"Killed the Emperor, nobody cared and got away ...",76561198089393905,2015,0
43,1250,True,"Compared to Left 4 Dead 2, this game REALLY gi...",DJKamBer,2013,0
55,238090,True,"One of the best sequals i have every seen, reb...",starkillershadow553,2015,0
56,242130,True,"If you're looking for a no frills, Arcade/Simu...",starkillershadow553,2016,0
...,...,...,...,...,...,...
58394,391460,False,Bad,iwishihadaids,2016,0
58404,381210,True,"Fix your ♥♥♥♥ing game, de rank me for no reaso...",76561198270958927,2016,0
58413,413150,True,"It's pretty fun, very very similiar to harvest...",vinquility,2016,0
58415,304930,True,I really like Unturned but the thing that i wa...,76561198289386531,2016,0


Vemos la tabla de outputgames y sacamos el nombre y el idtem solamente

In [65]:
# Vemos el dataset y sus columnas
output_games.head(1)

,Genres,ItemName,ReleaseDate,Price,ItemId,Developer,ReleaseYear
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018.0


In [66]:
# nos quedamos con itemid para hacer merge y con el itemname
columnas_seleccionadas = ['ItemId', 'ItemName']
output_games2 = output_games[columnas_seleccionadas]

In [67]:
# Vemos el dataset creado
output_games2.head()

,ItemId,ItemName
0,761140,Lost Summoner Kitty
1,643980,Ironbound
2,670290,Real Pool 3D - Poolians
3,767400,弹炸人2222
4,773570,Log Challenge


In [68]:
# Elegimos las columnas de user_reviews
user_reviews.columns

Index(['ItemId', 'Recommend', 'Review', 'UserId', 'YearPosted',
       'sentiment_analysis'],
      dtype='object')

In [69]:
# vemos la cantidad de datos en user_reviews
len(user_reviews)

58431

In [70]:
# Seleccionamos las columnas de cada DataFrame
columnas_file1 = output_games2[['ItemId','ItemName']]
columnas_file2 = user_reviews[['ItemId', 'Recommend','Review','UserId','YearPosted','sentiment_analysis']]
# Traemos la columna ItemId para hacer la referencia entre las tablas

In [71]:
# Merge basado en el campo ItemId
merged_reviews = pd.merge(columnas_file1, columnas_file2, left_on='ItemId', right_on='ItemId', how='inner')

In [72]:
# vemos como quedo en nuevo dataset
merged_reviews.head()

,ItemId,ItemName,Recommend,Review,UserId,YearPosted,sentiment_analysis
0,282010,Carmageddon Max Pack,True,"Oh, where do i start? DOS...",InstigatorAU,2016,1
1,70,Half-Life,True,ADVERTENCIA:Este Análisis contiene posibles Sp...,EizanAratoFujimaki,2015,2
2,70,Half-Life,True,If you own any half life but not this one that...,GamerFag,2011,0
3,70,Half-Life,True,this game has had me hooked for over 4 years a...,76561198020928326,2014,2
4,70,Half-Life,True,"This is, by far my most favourite game I have ...",Bluegills,2013,2


In [73]:
len(merged_reviews)

53196

In [74]:
# Ahora si vamos a tomar las columnas que precisamos para responder a la funcion
merged_reviews = merged_reviews.drop(['ItemId', 'UserId','Review'], axis=1)

In [75]:
# Lo vemos
merged_reviews.head(1)

,ItemName,Recommend,YearPosted,sentiment_analysis
0,Carmageddon Max Pack,True,2016,1


In [76]:
# vemos la informacion de nuestra tabla con el drop
merged_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53196 entries, 0 to 53195
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ItemName            53196 non-null  object
 1   Recommend           53196 non-null  bool  
 2   YearPosted          53196 non-null  int32 
 3   sentiment_analysis  53196 non-null  int64 
dtypes: bool(1), int32(1), int64(1), object(1)
memory usage: 1.5+ MB


Por ultimo vamos a hacer las agrupaciones para tenes la informacion que se pide en la funcion

In [77]:
# Filtrar por Recommend = True
df_filtered = merged_reviews[merged_reviews['Recommend'] == True]

In [78]:
# Agrupar por ItemName y contar la columna sentiment_analysis y mantener YearPosted
df_grouped = df_filtered.groupby('ItemName').agg({'sentiment_analysis': 'count', 'YearPosted': 'first'}).reset_index()

In [79]:
# Renombrar la columna resultante
df_grouped.rename(columns={'sentiment_analysis': 'total_sentiment_analysis'}, inplace=True)

In [80]:
df_grouped

,ItemName,total_sentiment_analysis,YearPosted
0,//N.P.P.D. RUSH//- The milk of Ultraviolet,1,2015
1,0RBITALIS,1,2014
2,"10,000,000",3,2015
3,100% Orange Juice,12,2015
4,100% Orange Juice - Krila & Kae Character Pack,1,2016
...,...,...,...
2864,the static speaks my name,13,2015
2865,theBlu,1,2016
2866,theHunter Classic,32,2015
2867,theHunter: Primal,13,2015


Como ultimo paso guardamos el dataset para la consulta

In [81]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2869 entries, 0 to 2868
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ItemName                  2869 non-null   object
 1   total_sentiment_analysis  2869 non-null   int64 
 2   YearPosted                2869 non-null   int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 56.2+ KB


In [82]:
# Guardamos el resultado en un nuevo archivo CSV
df_grouped.to_csv('consulta3.csv', index=False)

----------------------------------------------------------------

4.    def UsersWorstDeveloper( año : int ): Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Vamos a ver cuales son las tablas y columnas que precisamos para responder esta pregunta:

recommend (user_reviews)

sentiment_analysis (user_reviews)

developer (output_games)

Vamos a reutilizar la base de user_reviews que ya contiene el analisis de sentimiento hecho para la funcion numero 3

In [83]:
user_reviews.head()

,ItemId,Recommend,Review,UserId,YearPosted,sentiment_analysis
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011,2
1,22200,True,It's unique and worth a playthrough.,76561197970982479,2011,2
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2011,2
3,251610,True,I know what you think when you see this title ...,js41637,2014,2
4,227300,True,For a simple (it's actually not all that simpl...,js41637,2013,2


In [84]:
# 'columnas_a_eliminar' es una lista de nombres de columnas que vamos a eliminar
columnas_a_eliminar = ['Review', 'UserId']

# Eliminamos las columnas del DataFrame 'user_reviews' que no vamos a usar
user_reviews2 = user_reviews.drop(columnas_a_eliminar, axis=1)

In [85]:
# vemos como nos quedo
user_reviews2.head()

,ItemId,Recommend,YearPosted,sentiment_analysis
0,1250,True,2011,2
1,22200,True,2011,2
2,43110,True,2011,2
3,251610,True,2014,2
4,227300,True,2013,2


In [86]:
# Filtramos el DataFrame para traer las condiciones de la funcion(reviews.recommend = False y comentarios negativos)
filtro = (user_reviews2['Recommend'] == False) & (user_reviews2['sentiment_analysis'] == 0)
user_reviews2 = user_reviews2[filtro]


In [87]:
# vemos como quedo
user_reviews2.head()

,ItemId,Recommend,YearPosted,sentiment_analysis
62,359320,False,2015,0
132,344760,False,2015,0
183,437220,False,2016,0
226,299740,False,2016,0
241,48240,False,2014,0


In [88]:
# vemos cuantos datos tenemos ahora con el filtro
user_reviews2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2880 entries, 62 to 58394
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   ItemId              2880 non-null   int32
 1   Recommend           2880 non-null   bool 
 2   YearPosted          2880 non-null   int32
 3   sentiment_analysis  2880 non-null   int64
dtypes: bool(1), int32(2), int64(1)
memory usage: 70.3 KB


Preparamos el dataset con una agrupacion para contar la cantidad de reviews negativas y tambien agrupamos por año

In [89]:
user_reviews2 = user_reviews2.drop(columns='Recommend')

In [90]:
user_reviews2

,ItemId,YearPosted,sentiment_analysis
62,359320,2015,0
132,344760,2015,0
183,437220,2016,0
226,299740,2016,0
241,48240,2014,0
...,...,...,...
58265,295110,2016,0
58292,305920,2015,0
58341,570,2016,0
58351,370240,2015,0


In [91]:
# podemos agrupar y contar la cantidad de reviews negativas por año
grouped_reviews2 = user_reviews2.groupby(['YearPosted', 'ItemId']).agg({'sentiment_analysis': 'count'}).reset_index()

In [92]:
# Renombrar la columna resultante
grouped_reviews2 = grouped_reviews2.rename(columns={'sentiment_analysis': 'CountSentiment'})

In [93]:
# como quedo el dataframe
grouped_reviews2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1349 entries, 0 to 1348
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   YearPosted      1349 non-null   int64
 1   ItemId          1349 non-null   int64
 2   CountSentiment  1349 non-null   int64
dtypes: int64(3)
memory usage: 31.7 KB


In [94]:
grouped_reviews2.head(5)

,YearPosted,ItemId,CountSentiment
0,2011,440,1
1,2011,18700,2
2,2011,63940,1
3,2011,91310,1
4,2011,105400,1


In [95]:
# ahora vamos a traer el dataset de output_games, para traer el dato del desarrollador y agregarlo en la tabla
output_games_columnasf4 = output_games[['ItemId', 'Developer']]

In [96]:
output_games_columnasf4.head()

,ItemId,Developer
0,761140,Kotoshiro
1,643980,Secret Level SRL
2,670290,Poolians.com
3,767400,彼岸领域
4,773570,Desconocido


Pro ultimo vamos a juntar los datasets en un merge, para tener todos los datos disponibles para la consulta

In [97]:
# Combinamos los DataFrames en base a la columna 'ItemId'
merged_df = pd.merge(grouped_reviews2, output_games_columnasf4[['ItemId', 'Developer']], on='ItemId', how='left')

In [98]:
# Quitamos nulos
merged_df.dropna()

,YearPosted,ItemId,CountSentiment,Developer
0,2011,440,1,Valve
1,2011,18700,2,Broken Rules
2,2011,63940,1,1C Company
5,2012,440,1,Valve
7,2012,42920,1,NeoCoreGames
...,...,...,...,...
1342,2016,480630,1,Bethesda Game Studios
1343,2016,488280,1,AZAMATIKA
1345,2016,493370,1,Playsaurus
1346,2016,501760,1,Kiddy


In [99]:
# Este es el dataframe que vamos a utilizar para la funcion que nos pide
merged_df.head()

,YearPosted,ItemId,CountSentiment,Developer
0,2011,440,1,Valve
1,2011,18700,2,Broken Rules
2,2011,63940,1,1C Company
3,2011,91310,1,NaN
4,2011,105400,1,NaN


In [100]:
# Encontre que en el dataframe hay desarrolladores que no tienen el nombre, estan como nan o aparece 'Desconocido' 
# en la columna 'Developer'
unknown_count = merged_df['Developer'].value_counts().get('Desconocido', 0)

print(f"Cantidad de 'Desconocido': {unknown_count}")

Cantidad de 'Desconocido': 35


In [101]:
# Para la funcion los vamos a quitar
merged_df = merged_df.loc[merged_df['Developer'] != 'Desconocido']

In [102]:
# Dropeamos los nulos
merged_df.dropna()

,YearPosted,ItemId,CountSentiment,Developer
0,2011,440,1,Valve
1,2011,18700,2,Broken Rules
2,2011,63940,1,1C Company
5,2012,440,1,Valve
7,2012,42920,1,NeoCoreGames
...,...,...,...,...
1342,2016,480630,1,Bethesda Game Studios
1343,2016,488280,1,AZAMATIKA
1345,2016,493370,1,Playsaurus
1346,2016,501760,1,Kiddy


Guardamos el dataset para consumir en la consulta

In [103]:
# Guardamos el resultado en un nuevo archivo CSV
merged_df.to_csv('consulta4.csv', index=False)

--------------------------------------------------

5.    def sentiment_analysis( empresa desarrolladora : str ): Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}

Necesitamos las siguientes bases y columnas:

Developer (output_games)

sentiment_analysis (user_reviews)

In [104]:
# Primero chequeamos los datasets que necesitamos (user_reviews, output_games)
user_reviews.head()

,ItemId,Recommend,Review,UserId,YearPosted,sentiment_analysis
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011,2
1,22200,True,It's unique and worth a playthrough.,76561197970982479,2011,2
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2011,2
3,251610,True,I know what you think when you see this title ...,js41637,2014,2
4,227300,True,For a simple (it's actually not all that simpl...,js41637,2013,2


In [105]:
output_games.head()

,Genres,ItemName,ReleaseDate,Price,ItemId,Developer,ReleaseYear
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018.0
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,0.00,643980,Secret Level SRL,2018.0
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,0.00,670290,Poolians.com,2017.0
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域,2017.0
4,"[Action, Indie, Casual, Sports]",Log Challenge,NaT,2.99,773570,Desconocido,NaN


Ahora vamos a quitar las columnas que no precisamos para la funcion consulta

In [106]:
# Lista de columnas a eliminar
columns_to_drop = ['Genres', 'ItemName','ReleaseDate','Price']

# Eliminar las columnas especificadas
output_games_f5 = output_games.drop(columns=columns_to_drop)


In [107]:
output_games_f5.head(1)

,ItemId,Developer,ReleaseYear
0,761140,Kotoshiro,2018.0


In [108]:
output_games_f5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31994 entries, 0 to 31993
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ItemId       31994 non-null  int32  
 1   Developer    31994 non-null  object 
 2   ReleaseYear  29686 non-null  float64
dtypes: float64(1), int32(1), object(1)
memory usage: 625.0+ KB


In [109]:
# Lista de columnas a eliminar
columns_to_drop = ['Recommend', 'Review','UserId','YearPosted']

# Eliminar las columnas especificadas
user_reviews_f5 = user_reviews.drop(columns=columns_to_drop)

In [110]:
user_reviews_f5.head(1)

,ItemId,sentiment_analysis
0,1250,2


In [111]:
user_reviews_f5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   ItemId              58431 non-null  int32
 1   sentiment_analysis  58431 non-null  int64
dtypes: int32(1), int64(1)
memory usage: 684.9 KB


Vamos a hacer un merge de las dos tablas

In [112]:
# Realizar el merge por la columna 'ItemId'
merged_df = pd.merge(user_reviews_f5, output_games_f5, on='ItemId', how='inner')

In [113]:
merged_df.head()

,ItemId,sentiment_analysis,Developer,ReleaseYear
0,1250,2,Tripwire Interactive,2009.0
1,1250,2,Tripwire Interactive,2009.0
2,1250,0,Tripwire Interactive,2009.0
3,1250,1,Tripwire Interactive,2009.0
4,1250,1,Tripwire Interactive,2009.0


In [114]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53196 entries, 0 to 53195
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ItemId              53196 non-null  int32  
 1   sentiment_analysis  53196 non-null  int64  
 2   Developer           53196 non-null  object 
 3   ReleaseYear         49618 non-null  float64
dtypes: float64(1), int32(1), int64(1), object(1)
memory usage: 1.8+ MB


Vamos a pasar el dataframe a un csv para alimentar la funcion consulta

In [115]:
# Guardamos el resultado en un nuevo archivo CSV
merged_df.to_csv('consulta5.csv', index=False)

--------------------------------------------------------------------------------------------

para la funcion de machine learning vamos a preparar el dataset cuando hagamos el modelo ML (3-Modelo-ML.ipynb)